In [1]:
import sys
import os

# Set project root as module path
sys.path.append(os.path.abspath(".."))

In [2]:
# load api keys from .env file 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Setup: Import all modules
from src.chunking import load_and_chunk

from src.embedding import get_embedding_model
from src.vector_store import build_vectorstore, save_vectorstore, load_vectorstore
from src.query_api import build_qa_chain


/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Step 1: Load and chunk PDF
pdf_path = "../data/study_plan.pdf"  # change path if needed
chunks = load_and_chunk(pdf_path)
print(f"Total chunks: {len(chunks)}")
print(chunks[0])


Total chunks: 5
page_content='Study Plan
Ying Lu
1. Introduction of my background
I ﬁnished my undergraduate degree in both statistics and computer science at the University of
Minnesota in May 2021. I decided to continue studying computer science as a master student
at the University of Minnesota. Now I’m a second year graduate student at the University of
Minnesota.
2. Reasons why I chose to study in the UMN
University of Minnesota is a prestigious school and has numerous cutting-edge research con-' metadata={'producer': 'pdfTeX-1.40.20', 'creator': 'TeX', 'creationdate': '2023-01-04T22:34:51+08:00', 'moddate': '2023-01-04T22:34:51+08:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019) kpathsea version 6.3.1', 'source': '../data/study_plan.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


In [5]:
# Step 2: Build embedding model
embedding_model = get_embedding_model()

/Users/yinglu/Desktop/projects/pdf-rag-qa/src/embedding.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(


In [6]:
# Step 3: Build vector store
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks]
vectorstore = build_vectorstore(texts, metadatas, embedding_model=embedding_model)


/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
# Step 4: Save vector store locally
store_path = "../faiss_store"
save_vectorstore(vectorstore, path=store_path)

In [8]:
# Step 5: Load vector store and build QA chain
vs = load_vectorstore(path=store_path, embedding_model=embedding_model)


In [9]:
qa_chain = build_qa_chain(vs)

# Step 6: Ask a question and get answer
query = "What is this document about?"  # replace with your own
#print(qa_chain.prompt.format_prompt(question=query, context="...").to_string())

result = qa_chain(query)

print("Answer:\n", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(" -", doc.metadata)

Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/Users/yinglu/Desktop/projects/pdf-rag-qa/src/query_api.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/var/folders/12/_f4h5yr16lq5p9l1tf6zpgq80000gn/T/ipykernel_92573/4229599140.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)
/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: F

Answer:
 Ying Lu 1. Introduction of my background I finished my undergraduate degree in both statistics and computer science at the University of Minnesota in May 2021. I decided to continue studying computer science as a master student at the University of Minnesota. Now I'm a second year graduate student at the University of Minnesota.

Sources:
 - {'producer': 'pdfTeX-1.40.20', 'creator': 'TeX', 'creationdate': '2023-01-04T22:34:51+08:00', 'moddate': '2023-01-04T22:34:51+08:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019) kpathsea version 6.3.1', 'source': '../data/study_plan.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
 - {'producer': 'pdfTeX-1.40.20', 'creator': 'TeX', 'creationdate': '2023-01-04T22:34:51+08:00', 'moddate': '2023-01-04T22:34:51+08:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019) kpathsea version 6.3.1', 'source': '../data/study_plan.pdf'